<a href="https://colab.research.google.com/github/alexcarvajal/ModeloPrediccionBRT/blob/main/Generaci%C3%B3nModeloDatasetTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import joblib
import time
import psutil


data1 = pd.read_csv('datasetTool1mes.csv')

# Preprocesamiento de datos
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X = data1.drop(columns=['Inputs']).values
y = data1['Inputs'].values

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Función para reestructurar los datos para el LSTM
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# Aplicando la función create_dataset con los time_steps optimizados
time_steps = 9
X_lstm, y_lstm = create_dataset(X_scaled, y_scaled, time_steps=time_steps)

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=100))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=2)

# Guardar el modelo y los escaladores
model.save('modelo_entrenado.h5')
joblib.dump(scaler_X, 'scaler_X.pkl')
joblib.dump(scaler_y, 'scaler_y.pkl')

process = psutil.Process()
print(f"Memory usage: {process.memory_info().rss / (1024 * 1024)} megabytes")

end_time = time.time()
execution_time = end_time - start_time

# Calcular minutos y segundos
minutes = int(execution_time // 60)
seconds = int(execution_time % 60)

# Formatear y mostrar el tiempo de ejecución
print(f"Tiempo de ejecución: {minutes:02d}:{seconds:02d}")